In [ ]:
import os
print(os.getcwd())
import sys
sys.path.append('/local/scratch/kchan76/sparse-coarse-operator/')
from libs.pde import *
from libs.rotated_laplacian import RotatedLaplacian
from libs.models import BasicNet, MultiHeadedAttention
import torch

device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
print('device:', device)

# 3-Level Attention 

In [4]:
# seed = 63
list_eps = [1/100]
low_pi = 3
high_pi = 4
theta_low = np.pi/12*low_pi
theta_high = np.pi/12*high_pi
npts = 5
list_theta = np.linspace(theta_low, theta_high, npts)

In [3]:
train_grid_size = 31
k2 = 15
k3 = 15
same_vecs = False
PDE = RotatedLaplacian(train_grid_size,[k2,k3],list_eps, list_theta,'fixed xi',same_vecs=same_vecs)

In [4]:
# LEVEL 2
seed = 23
print('seed:', seed)
set_global_seed(seed)
h = 8
dhid = 256
epochs = 1500
adam_decay_rate = 0 # adam weight decay
lr = 1e-3 # learning rate
lr_decay_rate = 0.7 # learning rate decay
lr_decay_step = 500 # learning rate decay step size
init = 'xavier uniform' # "<name> <distribution>" name={'xavier', 'kaiming'} distribution={'uniform', 'normal'}

ncopy = 1
single_model = False # KEEP IT FALSE!! if true, only model_prob will be learned
softmax_topk = True
dropout = 0.3
enforce_stencil_symmetry = '+2'

model_prob_l2 = MultiHeadedAttention(h, dhid, indim=2, outdim=2, ncopy=ncopy, init=init, dropout=dropout)
model_value_l2 = MultiHeadedAttention(h, dhid, indim=2, outdim=2, ncopy=ncopy, init=init, dropout=dropout)


model_prob_l2,model_value_l2 = PDE.train(PDE.A2_train, 
                                         PDE.s2_train,
                                         PDE.eig_vec2_train,
                                         model_prob_l2,
                                         model_value_l2,
                                         epochs,
                                         adam_decay_rate,
                                         lr,
                                         lr_decay_rate,
                                         lr_decay_step,
                                         device,
                                         single_model=single_model,
                                         softmax_on = softmax_topk,
                                         enforce_stencil_symmetry = enforce_stencil_symmetry,
                                         verbose = True)




seed: 23


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:1! (when checking arugment for argument mat2 in method wrapper_mm)

In [6]:
# test level-2 model
test_seed = 13
# test_seed = test_seed+1
# for test_seed in range(50):
set_global_seed(test_seed)
print('test seed:', test_seed)

# sizes = [63,127,255,511]
sizes = [511]
num_test = 10
models = {}
models['level2'] = model_prob_l2, model_value_l2
test_grid_size = 63
max_levels = 2
epsilon = tuple(list_eps)
theta = (theta_low, theta_high)
accel = None
dropout_on = False
enforce_stencil_symmetry = None

for test_grid_size in sizes:
    print('size:', test_grid_size)
    set_global_seed(test_seed)
    num_iter_standard, num_iter_learning, thetaList, xs, xl = PDE.test_model(num_test,
                                                            models,
                                                            test_grid_size,
                                                            max_levels,
                                                            epsilon,
                                                            theta,
                                                            device,
                                                            dropout_on=dropout_on,
                                                            random_test = True,
                                                            single_model=single_model,
                                                            enforce_stencil_symmetry = enforce_stencil_symmetry,
                                                            softmax_on = softmax_topk,
                                                            top_accel = accel,
                                                            verbose = True)

test seed: 13
size: 511
standard stencil iter:    9.3   standard stencil time:   1.9372130393981934
learned stencil iter:    102.3   learned stencil time:   13.713903141021728


In [69]:
print(num_iter_standard)
print(num_iter_learning)

[12, 12, 12, 12, 12, 12, 12, 12, 12, 12]
[99, 254, 255, 176, 186, 208, 247, 217, 137, 237]


In [83]:
print(thetaList)

[2.1936428642673165, 2.242861055382838, 2.250320857584587, 2.2121678867095524, 2.214042612893607, 2.175985142843968, 2.17642663172932, 2.1715891287701963, 2.23335100780533, 2.1981257850804647]


In [14]:
print(xl)

[0.61980851 1.44330889 2.3055701  ... 0.85026996 0.2907676  0.19303396]


## Level 3

In [69]:
# LEVEL 3
l3_seed = 62
h3 = 8
dhid3 = 256
epochs3 = 1500
adam_decay_rate3 = 0 # adam weight decay
lr3 = 1e-4 # learning rate
lr_decay_rate3 = 1 # learning rate decay
lr_decay_step3 = 750 # learning rate decay step size
init3 = 'xavier uniform' 

ncopy3 = 1
single_model3 = False
softmax_topk3 = True
dropout3 = 0.5
enforce_stencil_symmetry = None


model_prob_l3 = MultiHeadedAttention(h3 ,dhid3, ncopy=ncopy3, init=init3, dropout=dropout3)
model_value_l3 = MultiHeadedAttention(h3, dhid3, ncopy=ncopy3, init=init3, dropout=dropout3)

model_prob_l3,model_value_l3 = PDE.train(PDE.A3_train, 
                                         PDE.s3_train,
                                         PDE.eig_vec3_train,
                                         model_prob_l3,
                                         model_value_l3,
                                         epochs3,
                                         adam_decay_rate3,
                                         lr3,
                                         lr_decay_rate3,
                                         lr_decay_step3,
                                         device,
                                         single_model= single_model3,
                                         softmax_on = softmax_topk3,
                                         enforce_stencil_symmetry = enforce_stencil_symmetry,
                                         verbose = True)

seed: 62
 epoch:  0  loss:  tensor(0.3078, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
 epoch:  250  loss:  tensor(0.0098, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
 epoch:  500  loss:  tensor(0.0070, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
 epoch:  750  loss:  tensor(0.0069, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
 epoch:  1000  loss:  tensor(0.0069, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
 epoch:  1250  loss:  tensor(0.0067, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
 epoch:  1499  loss:  tensor(0.0067, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)


In [41]:
# MODEL_DIR = f"./models/laplacian/fixed-xi/xi-{int(1/list_eps[0])}/theta-{low_pi}pi\\12-{high_pi}pi\\12/"

In [42]:
# MODEL_DIR = f"./models/laplacian/fixed-xi/xi-{int(1/list_eps[0])}/theta-{low_pi}pi\\12-{high_pi}pi\\12/"
# if not os.path.isdir(MODEL_DIR):
#     os.makedirs(MODEL_DIR)
# torch.save(model_prob_l2, MODEL_DIR+"level2_prob.pt")
# torch.save(model_value_l2, MODEL_DIR+"level2_value.pt")
# torch.save(model_prob_l3, MODEL_DIR+"level3_prob.pt")
# torch.save(model_value_l3, MODEL_DIR+"level3_value.pt")

In [43]:
# MODEL_DIR = f"./models/laplacian/fixed-xi/xi-{int(1/list_eps[0])}/theta-{low_pi}pi\\12-{high_pi}pi\\12/"
# model_prob_l2=torch.load(MODEL_DIR+"level2_prob.pt")
# model_value_l2=torch.load(MODEL_DIR+"level2_value.pt")
# model_prob_l3=torch.load(MODEL_DIR+"level3_prob.pt")
# model_value_l3 = torch.load(MODEL_DIR+"level3_value.pt")

In [76]:
test_seed = 14
# seed_list_127 =[]
# for test_seed in range(100):
set_global_seed(test_seed)
print('test seed:', test_seed)
sizes = [63]
num_test = 10
models = {}
models['level2'] = model_prob_l2, model_value_l2
models['level3'] = model_prob_l3, model_value_l3
test_grid_size = 127
max_levels = 3
epsilon = tuple(list_eps)
theta = (theta_low, theta_high)
random_test = True
dropout_on = False
accel = None
enforce_stencil_symmetry = None

for test_grid_size in sizes:
    print('size:',test_grid_size)
    set_global_seed(test_seed)
    num_iter_standard, num_iter_learning, thetaList, _, _, _, _, _, _ = PDE.test_model(num_test,
                                                            models,
                                                            test_grid_size,
                                                            max_levels,
                                                            epsilon,
                                                            theta,
                                                            device,
                                                            top_accel = accel,
                                                            dropout_on=dropout_on,
                                                            random_test=random_test,
                                                            single_model=single_model,
                                                            softmax_on = softmax_topk,
                                                            enforce_stencil_symmetry = enforce_stencil_symmetry,
                                                            verbose = True)
#     if np.mean(num_iter_learning) < np.mean(num_iter_standard):
#         seed_list_127.append(test_seed)
#         print("seed list:", seed_list_127)

test seed: 14
size: 63
standard stencil iter:    88.0   standard stencil time:   0.03599677085876465
learned stencil iter:    1001.0   learned stencil time:   0.8627280235290528


In [22]:
print(num_iter_standard)
print(num_iter_learning)

[12, 11, 12, 12, 12, 12, 12, 12, 12, 12]
[24, 38, 27, 32, 33, 32, 26, 25, 25, 23]


In [44]:
thetaList

[1.7930162802928942,
 1.6176311704189765,
 1.5850285830191142,
 1.6654468706466492,
 1.6428961213033724,
 1.7095500612059304,
 1.650885711650739,
 1.6505075276219896,
 1.6000501244895629,
 1.6362197356403216]

In [71]:
from pyamg.gallery import diffusion_stencil_2d
from scipy.io import savemat

file_ind = 0

MODEL_DIR = f"./matrix/xi-{int(1/list_eps[0])}/theta-{low_pi}pi_12-{high_pi}pi_12"
if not os.path.isdir(MODEL_DIR):
    os.makedirs(MODEL_DIR)
    
for theta in thetaList:
    s = np.double(diffusion_stencil_2d(epsilon=int(1/list_eps[0]), theta=theta, type='FD'))
    A = pyamg.gallery.stencil_grid(s,(511,511)).tocsr()
    filename = f"./matrix/xi-{int(1/list_eps[0])}/theta-{low_pi}pi_12-{high_pi}pi_12/{file_ind}.mat"
    savemat(filename, {'A':A})
    file_ind += 1

In [11]:
MODEL_DIR = f"./matrix/xi-{int(1/list_eps[0])}/theta_{low_pi}pi_12-{high_pi}pi_12"
if not os.path.isdir(MODEL_DIR):
    os.makedirs(MODEL_DIR)

# MLP

In [72]:
# define problem
list_eps = [0.01]
theta_low = np.pi/6
theta_high = np.pi/4
list_theta = np.linspace(theta_low, theta_high, 10)

In [74]:
# seed += 1
print('seed:',seed)
set_global_seed(seed)
epochs = 2000
adam_decay_rate = 0 # adam weight decay
lr = 1e-3 # learning rate
lr_decay_rate = 0.5 # learning rate decay
lr_decay_step = 500 # learning rate decay step size
model_prob_l2 = BasicNet([64])
model_value_l2 = BasicNet([64])
k2 = 10
k3 = 15
same_vecs = False
single_model = False # if true, only model_prob will be learned. 
softmax_on = True # if true, turn on the softmax in the function top_k during sparsfication
problem = RotatedLaplacian(31,[k2,k3],list_eps, list_theta,'fixed xi',same_vecs=same_vecs)

model_prob_l2,model_value_l2 = problem.train(problem.A2_train, 
                                             problem.s2_train,
                                             problem.eig_vec2_train,
                                             model_prob_l2,
                                             model_value_l2,
                                             epochs,
                                             adam_decay_rate,
                                             lr,
                                             lr_decay_rate,
                                             lr_decay_step,
                                             device,
                                             single_model=single_model,
                                             softmax_on = softmax_on,
                                             verbose = True)

seed: 36
 epoch:  0  loss:  tensor(6.7295, device='cuda:0', grad_fn=<AddBackward0>)
 epoch:  250  loss:  tensor(0.0111, device='cuda:0', grad_fn=<AddBackward0>)
 epoch:  500  loss:  tensor(0.0104, device='cuda:0', grad_fn=<AddBackward0>)
 epoch:  750  loss:  tensor(0.0101, device='cuda:0', grad_fn=<AddBackward0>)
 epoch:  1000  loss:  tensor(0.0098, device='cuda:0', grad_fn=<AddBackward0>)
 epoch:  1250  loss:  tensor(0.0096, device='cuda:0', grad_fn=<AddBackward0>)
 epoch:  1500  loss:  tensor(0.0095, device='cuda:0', grad_fn=<AddBackward0>)
 epoch:  1750  loss:  tensor(0.0094, device='cuda:0', grad_fn=<AddBackward0>)
 epoch:  1999  loss:  tensor(0.0093, device='cuda:0', grad_fn=<AddBackward0>)


In [11]:
stncl = np.array([[ 0,  0.0510,  0],
          [ 0.0207, -0.0747, 0.0207],
          [ 0,  0.0510, 0]])
pyamg.gallery.stencil_grid(stncl,(3,3)).toarray()

array([[-0.0747,  0.0207,  0.    ,  0.051 ,  0.    ,  0.    ,  0.    ,
         0.    ,  0.    ],
       [ 0.0207, -0.0747,  0.0207,  0.    ,  0.051 ,  0.    ,  0.    ,
         0.    ,  0.    ],
       [ 0.    ,  0.0207, -0.0747,  0.    ,  0.    ,  0.051 ,  0.    ,
         0.    ,  0.    ],
       [ 0.051 ,  0.    ,  0.    , -0.0747,  0.0207,  0.    ,  0.051 ,
         0.    ,  0.    ],
       [ 0.    ,  0.051 ,  0.    ,  0.0207, -0.0747,  0.0207,  0.    ,
         0.051 ,  0.    ],
       [ 0.    ,  0.    ,  0.051 ,  0.    ,  0.0207, -0.0747,  0.    ,
         0.    ,  0.051 ],
       [ 0.    ,  0.    ,  0.    ,  0.051 ,  0.    ,  0.    , -0.0747,
         0.0207,  0.    ],
       [ 0.    ,  0.    ,  0.    ,  0.    ,  0.051 ,  0.    ,  0.0207,
        -0.0747,  0.0207],
       [ 0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.051 ,  0.    ,
         0.0207, -0.0747]])